In [1]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import RobustScaler
import joblib

print("CUDA:", torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"


CUDA: True


In [2]:
df = pd.read_parquet(
    "/kaggle/input/canonical-checkpoint-features-with-soh-and-split"
)

print(df.shape)
df.head()


(118770, 27)


,asset_id,cycle_id,V_mean,V_std,V_min,V_max,V_range,dV_dt_mean,dV_dt_max,T_mean,...,T_delta_base,duration_base,use_dV_dt,V_range_norm,T_delta_norm,duration_norm,dV_dt_norm,Degradation_Index,SOH_proxy,split
0,0,0,3.735076,0.245422,2.699819,4.191235,1.491416,-35.948592,38.111247,40.419340,...,1.335614,0.038896,True,0.692878,0.669490,0.729033,0.741807,0.708302,0.492480,train
1,0,1,3.735681,0.244972,2.699859,4.192679,1.492819,-36.342517,34.652440,40.327115,...,1.335614,0.038896,True,0.693348,0.688454,0.722873,0.693107,0.699446,0.496861,train
2,0,10,3.739525,0.240002,2.699924,4.192582,1.492658,-38.022390,41.584546,40.363359,...,1.335614,0.038896,True,0.693295,0.693133,0.699661,0.788435,0.718631,0.487419,train
3,0,11,3.739852,0.239651,2.699803,4.192502,1.492699,-38.328526,34.646588,40.433203,...,1.335614,0.038896,True,0.693308,0.711655,0.697407,0.693023,0.698848,0.497158,train
4,0,12,3.740231,0.238893,2.699924,4.192462,1.492538,-38.309224,34.655228,40.393250,...,1.335614,0.038896,True,0.693254,0.702437,0.695238,0.693147,0.696019,0.498566,train


In [3]:
FEATURES = [
    "V_mean","V_std","V_min","V_max","V_range",
    "dV_dt_mean","dV_dt_max",
    "T_mean","T_max","T_delta",
    "duration_s"
]

TARGET = "SOH_proxy"
WINDOW = 20


In [4]:
df = df.sort_values(
    ["asset_id", "cycle_id"]
).reset_index(drop=True)


In [5]:
df_train = df[df["split"] == "train"].copy()
print("Train rows:", len(df_train))


Train rows: 83134


In [6]:
feature_medians = df_train[FEATURES].median()
feature_medians


V_mean          3.313600
V_std           0.049963
V_min           3.200000
V_max           3.468000
V_range         0.170000
dV_dt_mean     -0.001260
dV_dt_max       0.000000
T_mean         27.856234
T_max          28.684860
T_delta         0.600660
duration_s    118.280000
dtype: float64

In [7]:
def sequence_generator(df, window, medians):
    for asset_id, g in df.groupby("asset_id"):
        g = g.reset_index(drop=True)

        if len(g) < window:
            continue

        X = g[FEATURES].fillna(medians).values.astype("float32")
        y = g[TARGET].values.astype("float32")

        for i in range(len(g) - window + 1):
            yield X[i:i+window], y[i+window-1]


In [8]:
from tqdm import tqdm

X_list, y_list = [], []

for x_seq, y_val in tqdm(sequence_generator(df_train, WINDOW, feature_medians)):
    X_list.append(x_seq)
    y_list.append(y_val)

X_train = torch.tensor(np.array(X_list), dtype=torch.float32)
y_train = torch.tensor(np.array(y_list), dtype=torch.float32)

print(X_train.shape, y_train.shape)


82925it [00:00, 633332.05it/s]


torch.Size([82925, 20, 11]) torch.Size([82925])


In [9]:
scaler = RobustScaler()

X2d = X_train.reshape(-1, X_train.shape[-1]).numpy()
scaler.fit(X2d)

X_scaled = scaler.transform(X2d)
X_train = torch.tensor(
    X_scaled.reshape(X_train.shape),
    dtype=torch.float32
)

joblib.dump(scaler, "lstm_scaler.joblib")


['lstm_scaler.joblib']

In [10]:
y_mean = y_train.mean()
y_train_centered = y_train - y_mean


In [11]:
train_loader = DataLoader(
    TensorDataset(X_train, y_train_centered),
    batch_size=32,
    shuffle=True
)


In [12]:
class SOHLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=11,
            hidden_size=128,
            batch_first=True
        )
        self.fc = nn.Sequential(
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1]).view(-1)


In [13]:
model = SOHLSTM().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
loss_fn = nn.MSELoss()


In [14]:
best_loss = float("inf")

for epoch in range(30):
    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)

        optimizer.zero_grad()
        preds = model(xb)
        loss = loss_fn(preds, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch:02d} | Train MSE: {avg_loss:.6f}")

    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), "soh_lstm_model.pt")


Epoch 00 | Train MSE: 0.000671
Epoch 01 | Train MSE: 0.000199
Epoch 02 | Train MSE: 0.000188
Epoch 03 | Train MSE: 0.000144
Epoch 04 | Train MSE: 0.000104
Epoch 05 | Train MSE: 0.000098
Epoch 06 | Train MSE: 0.000147
Epoch 07 | Train MSE: 0.000136
Epoch 08 | Train MSE: 0.000123
Epoch 09 | Train MSE: 0.000111
Epoch 10 | Train MSE: 0.000086
Epoch 11 | Train MSE: 0.000090
Epoch 12 | Train MSE: 0.000091
Epoch 13 | Train MSE: 0.000074
Epoch 14 | Train MSE: 0.000073
Epoch 15 | Train MSE: 0.000067
Epoch 16 | Train MSE: 0.000092
Epoch 17 | Train MSE: 0.000080
Epoch 18 | Train MSE: 0.000097
Epoch 19 | Train MSE: 0.000079
Epoch 20 | Train MSE: 0.000068
Epoch 21 | Train MSE: 0.000066
Epoch 22 | Train MSE: 0.000100
Epoch 23 | Train MSE: 0.000069
Epoch 24 | Train MSE: 0.000059
Epoch 25 | Train MSE: 0.000067
Epoch 26 | Train MSE: 0.000069
Epoch 27 | Train MSE: 0.000076
Epoch 28 | Train MSE: 0.000078
Epoch 29 | Train MSE: 0.000080
